In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# # Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install torchsummary

In [ ]:
import sys
sys.path.append('../')
from functions.load_data import MarielDataset, edges
from functions.functions import *
from functions.modules import *
from functions.plotting import *

import torch
from torch.utils.data import Dataset, Subset, SubsetRandomSampler, SequentialSampler
from torch.utils.data.dataset import TensorDataset
from torch.distributions.multivariate_normal import MultivariateNormal
import torch.nn.functional as F
from torch_geometric.data import DataLoader
from torch_geometric.data import Data
from torch_geometric.utils.convert import to_networkx
torch.manual_seed(0)

import networkx as nx # for visualizing graphs
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pdb
import json
import pickle
import os

# Test

In [ ]:
! ls -latrh ../logs/

In [ ]:
folder = "../logs/fulltraining_lr1e-6_sparsity"
dataloader_test = torch.load(os.path.join(folder,"dataloader_test.pth")) ## override with my longer dataset loader
checkpoint_path = os.path.join(folder,"best_weights.pth")
args_file = os.path.join(folder, 'args.pkl')
args = pickle.load(open(args_file, "rb" ))['args']
checkpoint_loaded = False 
print(args)

# Load these if training actually completed:
if os.path.exists(os.path.join(folder,"losses.json")):
    dict = json.load(open(os.path.join(folder,"losses.json")))
    train_losses = dict['train_losses']
    val_losses = dict['val_losses']
    
    fig, ax = plt.subplots(figsize=(8,6))
    ax.plot(np.arange(len(train_losses)), train_losses, label="Training")
    ax.plot(np.arange(len(val_losses)), val_losses, label="Validation")
    ax.set_xlabel("Epoch", fontsize=16)
    ax.set_ylabel("Loss", fontsize=16)
#     ax.set_yscale("log")
#     ax.set_ylim(-0.05,20)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    ax.legend(fontsize=14)
    plt.savefig(os.path.join(folder,"loss_curve.jpg"))

In [ ]:
if torch.cuda.is_available() and args.no_cuda == False:
    device = torch.device('cuda')
else:
    device = 'cpu'

model = NRI(device=device,
            node_features=args.seq_len*6, 
            edge_features=args.seq_len, 
            hidden_size=args.hidden_size, 
            node_embedding_dim=args.node_embedding_dim,
            edge_embedding_dim=args.edge_embedding_dim,
            skip_connection=args.skip_connection,
            dynamic_graph=False,
            seq_len=args.seq_len,
            predicted_timesteps=args.predicted_timesteps,
            ablation_edge_index=3,
            threshold=50, # use top (100-x)% of edges
           )

optimizer = torch.optim.Adam(list(model.parameters()), lr=args.lr, weight_decay=5e-4)

print("Using {}".format(device))
model = model.to(device)
# print(model)
print("Total trainable parameters: {:,}".format(count_parameters(model)))

In [ ]:
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss_checkpoint = checkpoint['loss']
checkpoint_loaded = True
n_joints = 53
print(loss_checkpoint)

In [ ]:
def test(batch_limit=0):
    mse_loss = torch.nn.MSELoss(reduction='mean')
    prediction_to_reconstruction_loss_ratio = 1 # CHANGE THIS TO ARGS
    total_test_loss = 0
    n_batches = 0
    actuals = []
    preds = []
    edge_types_list = []
    logits_list = []
    probabilities_list = []
    model.eval()
    
    for batch in tqdm(dataloader_test, desc="Test batches"):
        batch = batch.to(device)

        ### CALCULATE MODEL OUTPUTS
        output, edge_types, logits, edges = model(batch)
        
        ### SAVE FOR ANIMATIONS
        actuals.append(batch.x.detach().cpu().numpy())
        preds.append(output.detach().cpu().numpy())
        logits_list.append(logits.detach().cpu().numpy())
        edge_types_list.append(edge_types.detach().cpu().numpy())
        probabilities_list.append(edges.detach().cpu().numpy())

        ### CALCULATE LOSS
        test_loss = mse_loss(batch.x.to(device), output) # compare first seq_len timesteps

        ### ADD LOSSES TO TOTALS
        total_test_loss += test_loss.item()
            
        ### OPTIONAL -- STOP TESTING EARLY
        n_batches += 1
        if (batch_limit > 0) and (n_batches >= batch_limit): break # temporary -- for stopping training early

    ### CALCULATE AVERAGE LOSSES PER EPOCH   
    average_test_loss = total_test_loss / n_batches
    print("Loss = {:,.8f}".format(average_test_loss))
    return actuals, preds, edge_types_list, logits_list, probabilities_list

In [ ]:
actuals, preds, edge_types, logits, probs = test(batch_limit = 10)

In [ ]:
### ANIMATE A SINGLE BATCH ONLY 
batch_number = 9
truth_sequences= []
predicted_sequences = []

for seq_number in np.arange(args.batch_size):
    actual = actuals[batch_number][seq_number*n_joints:seq_number*n_joints+n_joints].reshape((n_joints,args.seq_len,6))[:,:,:3] # take the first 3 dimensions for positions, not velocities
    pred = preds[batch_number][seq_number*n_joints:seq_number*n_joints+n_joints].reshape((n_joints,args.seq_len,6))[:,:,:3]
    actual = np.transpose(actual, [1,0,2])
    pred = np.transpose(pred, [1,0,2])
    truth_sequences.append(actual)
    predicted_sequences.append(pred)
    
truth_sequences = np.asarray(truth_sequences).reshape((args.batch_size*args.seq_len, n_joints, 3))
predicted_sequences = np.asarray(predicted_sequences).reshape((args.batch_size*args.seq_len, n_joints, 3))

start_index = 0
# timesteps = args.seq_len*args.batch_size
timesteps = args.seq_len
animation = animate_stick(truth_sequences[start_index:start_index+timesteps,:,:], 
                          ghost=predicted_sequences[start_index:start_index+timesteps,:,:], 
                          ghost_shift=0.4,
                          ax_lims = (-0.7,0.3),
                          skeleton=True,
                          skeleton_alpha=1,
                          figsize=(10,8), cmap='inferno')
HTML(animation.to_html5_video())

Now do the same thing, but with edge types...

In [ ]:
edge_types = edge_types[0] # one-hot encoded edges
logits = logits[0] # unnormalized log-probabilities
probs = probs[0] # normalized logits

In [ ]:
np.sum(edge_types, axis=0)

In [ ]:
# percentage of the edges that belong to each class
percentages = edge_types.sum(axis=0)/edge_types.shape[0]

for i in range(len(percentages)):
    print("Edge type {}: {:.1f}%".format(i, percentages[i]*100))

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
bins = np.linspace(0.,1,100)
for i in range(4):
    ax.hist(probs[:,i],bins=bins, label="Edge type "+str(i), histtype="step", linewidth=3)
ax.legend(fontsize=15, loc="best")
ax.set_title("Edge Type Probabilities (Normalized)", fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=17)

In [ ]:
edge_class = 0
threshold = np.percentile(probs[:,edge_class],98) # top 1% of edges if the threshold is 99
print(threshold)
timesteps=1
animation = animate_stick(truth_sequences[start_index:start_index+timesteps,:,:], 
#                           ghost=predicted_sequences[start_index:start_index+timesteps,:,:], 
#                           ghost_shift=0.4,
                          cloud=True,
                          cloud_alpha=0.003, # default=0.03 or 0.1
                          edge_types=edge_types,
                          edge_opacities=probs, 
                          threshold=threshold, # only plot the edges above this threshold with opacities given by scaled argmax
                          edge_class = edge_class,
                          skeleton = True,
                          ax_lims = (-0.5,0.5),
                          figsize=(10,8), 
                          cmap='Greys',
                         )
HTML(animation.to_html5_video())

# Test w/ predictions

In [ ]:
test_seq_len = 100
data = MarielDataset(seq_len=test_seq_len, 
                     reduced_joints=False, 
                     predicted_timesteps=100-args.seq_len, 
                     no_overlap=True, 
                     file_path="../data/mariel_*.npy")
test_indices = np.arange(int(0.85*len(data)), len(data)) # last 15% on test
test = torch.utils.data.Subset(data, test_indices)
dataloader_test_long = DataLoader(test, batch_size=1, shuffle=False, drop_last=True)

In [ ]:
def test_pred(batch_limit=0):
    mse_loss = torch.nn.MSELoss(reduction='mean')
    total_test_loss = 0
    n_batches = 0
    actuals_all = []
    preds_all = []
    preds_all_for_animation = []
    model.eval()
    for batch in tqdm(dataloader_test_long, desc="Test batches"):
        actuals = []
        preds = []
        preds_for_animation = []
        batch = batch.to(device)
        for i in range(test_seq_len):
            if i == 0 :
                x = batch.x[:,:args.seq_len*6]
                input_seq = Data(x=x, edge_index=batch.edge_index, edge_attr=batch.edge_attr)
            elif i == 1: 
                preds_for_animation.append(preds[i-1]) # start out the animation of predictions with the real steps + first predictions
                x_real = batch.x[:,:(args.seq_len-args.predicted_timesteps)*6]
                x_pred = torch.tensor(preds[i-1][:,-args.predicted_timesteps*6:]).to(device) # add on the previous sequence's predictions
                x = torch.cat([x_real, x_pred], axis=1)
                input_seq = Data(x=x, edge_index=batch.edge_index, edge_attr=batch.edge_attr)
                preds_for_animation.append(x_pred.detach().cpu().numpy())
            elif i*args.predicted_timesteps < args.seq_len:
                x_real = batch.x[:,:(args.seq_len-i*args.predicted_timesteps)*6]
                new_pred = torch.tensor(preds[i-1][:,-args.predicted_timesteps*6:]).to(device)
                x_pred = torch.cat([x_pred, new_pred], axis=1) # add on the previous sequence's predictions
                x = torch.cat([x_real, x_pred], axis=1)
                input_seq = Data(x=x, edge_index=batch.edge_index, edge_attr=batch.edge_attr)
                preds_for_animation.append(new_pred.detach().cpu().numpy())
            elif i*args.predicted_timesteps >= args.seq_len:
                x = torch.tensor(preds[i-1]).to(device) # now this is the right size to feed into the model
                input_seq = Data(x=x, edge_index=batch.edge_index, edge_attr=batch.edge_attr)
                new_pred = torch.tensor(preds[i-1][:,-args.predicted_timesteps*6:]).to(device)
                preds_for_animation.append(new_pred.detach().cpu().numpy())
                
            ### CALCULATE MODEL OUTPUTS
            output, edge_types, logits, edges = model(input_seq)

            ## CALCULATE LOSS
            test_loss = mse_loss(input_seq.x, output) # compare first seq_len timesteps
    
            ### SAVE FOR ANIMATIONS  
            preds.append(output.detach().cpu().numpy())
    
        ### STACK
        actuals_all.append(batch.x.detach().cpu().numpy())
        preds_all.append(np.hstack(preds))
        preds_all_for_animation.append(np.hstack(preds_for_animation))
        
        ### ADD LOSSES TO TOTALS
        total_test_loss += test_loss.item()
            
        ### OPTIONAL -- STOP TESTING EARLY
        n_batches += 1
        if (batch_limit > 0) and (n_batches >= batch_limit): break # temporary -- for stopping training early

    ### CALCULATE AVERAGE LOSSES PER EPOCH   
    average_test_loss = total_test_loss / n_batches
    print("Loss = {:,.8f}".format(average_test_loss))
    return actuals_all, preds_all, preds_all_for_animation, test_seq_len, n_batches

In [ ]:
actuals, preds, preds_animation, test_seq_len, n_batches = test_pred(batch_limit = 1)

In [ ]:
### ANIMATE A SINGLE BATCH ONLY 
batch_number = 0
truth_sequences= []
predicted_sequences = []

pred_steps = int(preds_animation[0].shape[1]/6)

actual = actuals[batch_number].reshape((n_joints,test_seq_len,6))[:,:,:3] # take the first 3 dimensions for positions, not velocities
pred = preds_animation[batch_number].reshape((n_joints,pred_steps,6))[:,:,:3]
actual = np.transpose(actual, [1,0,2])
pred = np.transpose(pred, [1,0,2])
truth_sequences.append(actual)
predicted_sequences.append(pred)
    
truth_sequences = np.asarray(truth_sequences).reshape((test_seq_len, n_joints, 3))
predicted_sequences = np.asarray(predicted_sequences).reshape((pred_steps, n_joints, 3))

# timesteps = args.seq_len*args.batch_size
timesteps = pred_steps
animation = animate_stick(truth_sequences[:timesteps,:,:], 
                          ghost=predicted_sequences[:timesteps,:,:], 
                          ghost_shift=0.4,
                          ax_lims = (-0.7,0.3),
                          skeleton=True,
                          skeleton_alpha=1,
                          figsize=(10,8), cmap='inferno')
HTML(animation.to_html5_video())

# Bar plots

In [ ]:
seq0 = [0.00005295,0.00260335,0.00151683,0.00099490,0.00178975]
avg = [0.00006506,0.00261071,0.00157890,0.00101111,0.00178699]

labels = ["All edge types", "Only Type 0", "Only Type 1", "Only Type 2", "Only Type 3"]
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(12,8))
bins = np.linspace(0.,1,100)

rects1 = ax.bar(x - width/2, seq0, width, label="One test sequence")
rects2 = ax.bar(x + width/2, avg, width, label="Average of 25 test sequences")

# ax.set_ylabel('MSE')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=0)
ax.legend(fontsize=15, loc="best")
ax.set_title("Reconstruction Loss (MSE)", fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=17)

In [ ]:
seq0 = [0.00006674,0.00024483,0.00020931,0.00016127,0.00013859]


labels = ["All edge types", "Only Type 0", "Only Type 1", "Only Type 2", "Only Type 3"]
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(12,8))
bins = np.linspace(0.,1,100)

rects1 = ax.bar(x, seq0, width, label="One test sequence")

# ax.set_ylabel('MSE')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=0)
ax.legend(fontsize=15, loc="best")
ax.set_title("Reconstruction Loss (MSE)", fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=17)

In [ ]:
seq0 = [0.00015296,0.00063390,0.00037075,0.00053506,0.00045124]
top1percent = [0.00060024,0.00063390,0.00061622,0.00062297,0.00062868]
top50percent = [0.00016835,0.00063390,0.00037819,0.00054157,0.00046595]
labels = ["All edge types", "Only Type 0", "Only Type 1", "Only Type 2", "Only Type 3"]
x = np.arange(len(labels))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots(figsize=(12,8))
bins = np.linspace(0.,1,100)

ax.bar(x - width, seq0, width, label="One test sequence")
ax.bar(x, top50percent, width, label="Top 50% of Edges")
ax.bar(x + width, top1percent, width, label="Top 1% of Edges")

# ax.set_ylabel('MSE')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=0)
ax.legend(fontsize=15, loc="best")
ax.set_title("Reconstruction Loss (MSE)", fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=17)